In [ ]:
# See https://github.com/cmap/cmapPy/tree/master/tutorials for other examples of how to use the cmapPy library
# and explore this data.


%load_ext autoreload
%autoreload 2
D='/home/ubuntu/2xar/ws/downloads/lincs/LINCS2020/'

In [ ]:
from cmapPy.pandasGEXpress.parse import parse
import pandas as pd
sig_info = pd.read_csv(D+"siginfo_beta.txt", sep="\t")

In [ ]:
display(sig_info.columns)
sig_info

In [ ]:
cmp_info = pd.read_csv(D+"compoundinfo_beta.txt", sep="\t")
cmp_info

In [ ]:
gene_info = pd.read_csv(D+"geneinfo_beta.txt", sep="\t")
gene_info

In [ ]:
cmp_info

In [ ]:

vorinostat_ids = sig_info["sig_id"][sig_info["cmap_name"] == "vorinostat"]
vorinostat_ids

In [ ]:
# Requires the control dataset to be downloaded, currently not part of the ETL.
# ctl_col = parse(D+"level5_beta_ctl_n58022x12328.gctx", col_meta_only=True)
#display(ctl_col.index.tolist())
# ctl_row = parse(D+"level5_beta_ctl_n58022x12328.gctx", row_meta_only=True)
#display(ctl_row)


In [ ]:
#sig_info[sig_info["sig_id"].isin(ctl_col.index.tolist())]['cmap_name'].unique()

In [ ]:
sig_info[(sig_info["cmap_name"] == "DMSO")]

In [ ]:

DMSO_ids = sig_info["sig_id"][(sig_info["cmap_name"] == "DMSO") & (sig_info["pert_type"] == "ctl_vehicle")]
DMSO_ids

In [ ]:
vorin_data = parse(D+"level5_beta_trt_cp_n720216x12328.gctx", cid=vorinostat_ids)
vorin_data


In [ ]:
vorin_data.data_df.shape

In [ ]:
vorinostat_sig_id_info =  sig_info[sig_info["cmap_name"] == "vorinostat"]
# let's look at a couple rows! 
display(vorinostat_sig_id_info[0:5])
vorinostat_sig_id_info.set_index("sig_id", inplace=True)
# now the data frame is indexed by sig_ids consistent with those of the data_df:
vorin_data.col_metadata_df = vorinostat_sig_id_info
vorin_data.multi_index_df

In [ ]:
gene_info = pd.read_csv(D+"geneinfo_beta.txt", sep="\t")
gene_info.set_index('gene_id', inplace=True)
gene_info.index = gene_info.index.astype('str')
vorin_data.row_metadata_df = gene_info

In [ ]:
import cmapPy.pandasGEXpress.GCToo as GCToo

vorin_multi = GCToo.GCToo(data_df=vorin_data.data_df.copy(), 
                                              row_metadata_df=vorin_data.row_metadata_df.copy(), 
                                              col_metadata_df=vorin_data.col_metadata_df.copy(), 
                                             make_multiindex=True)

vorin_multi.multi_index_df

In [ ]:
vorin_multi.data_df.mean(axis=1).values

In [ ]:
df = vorin_multi.data_df
df

In [ ]:
sub_ids = vorinostat_ids[2:5]
df[sub_ids]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(go.Heatmap(
        z=df.values[:200,0:200],
        colorscale='Picnic'))
fig.show()

In [ ]:
from dtk.plot import plotly_heatmap

if False:
    row_labels = [str(x) for x in range(200)]
    pp = plotly_heatmap(
        df.values[:200, :200],
        row_labels,
        reorder_cols=True,
        width=1200,
        height=1200,
        colorscale='Picnic'
    )

    pp.as_figure().show()

In [ ]:
mdf = vorin_multi.multi_index_df
landmark_only = mdf.query("feature_space == 'landmark'")
landmark_only

In [ ]:

fig = go.Figure(go.Heatmap(
        z=landmark_only.values[:,0:200],
        colorscale='Picnic'))
fig.show()

In [ ]:
landmark_only = landmark_only.reindex(landmark_only.abs().mean(axis=0).sort_values().index, axis=1)
fig = go.Figure(go.Heatmap(
        z=landmark_only.values[:,0::5],
        colorscale='Picnic'))
fig.show()

In [ ]:
landmark_only = landmark_only.reindex(landmark_only.mean(axis=1).sort_values().index, axis=0)
fig = go.Figure(go.Heatmap(
        z=landmark_only.values[:,0::5],
        colorscale='Picnic'))
fig.show()

In [ ]:
one_cell = landmark_only.xs('A375', level='cell_iname', axis=1)
one_cell

In [ ]:
fig = go.Figure(go.Heatmap(
        z=one_cell.values,
        colorscale='Picnic'))
fig.show()

In [ ]:
col_ord = sorted(landmark_only.columns, key=lambda x: (x[2], x[6]) )
landmark_by_dose = landmark_only.reindex(col_ord, axis=1)
landmark_by_dose

In [ ]:
fig = go.Figure(go.Heatmap(
        z=landmark_by_dose.values[:, ::5],
        colorscale='Picnic'))
fig.show()